# Single-variable nonlinear program

We'll look at two types of solvers we can use to for nonlinear programs.  The first one, called IPOPT, you can think of like GRG Nonlinear in Excel.  You can install this solver by running the next cell (uncomment it first!).

In [ ]:
#!conda install -c conda-forge ipopt

Let's suppose we have a smooth, nonlinear function of one variable defined on the interval [-1, 1].

In [ ]:
import numpy as np

def nonlinear_function(x):
    return -((x - 0.5)**2 - 1)**2

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

x = np.linspace(-1, 1, 1000)
y = [nonlinear_function(xi) for xi in x]
_ = plt.scatter(x, y, marker='.')

We would like to find the point (-1, -1.5625) as the global minimum on this domain, but notice there is another local minimum at `x=0.5`.  Which one will the solver find?

Let's create the Pyomo model.

Solve using `ipopt`.

In [ ]:
import pyomo_helpers as helpers

instance, obj, solution = helpers.solve(model, solver='ipopt')